In [262]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gastonmontes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [263]:
# Leo los archivos a utilizar. Solo las columnas de mi interés.
trainData = pd.read_csv('../Data/train.csv', usecols=['id', 'text', 'target'])
testData = pd.read_csv('../Data/test.csv', usecols=['id', 'text'])

# Proceso el train y test set
Obtengo el train set como (word, target) y el test set como (id, word)

Me fijo cómo es la estructura de cada set de datos.

In [264]:
trainData.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [265]:
testData.tail()

,id,text
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,Storm in RI worse than last hurricane. My city...
3260,10868,Green Line derailment in Chicago http://t.co/U...
3261,10874,MEG issues Hazardous Weather Outlook (HWO) htt...
3262,10875,#CityofCalgary has activated its Municipal Eme...


## Proceso el train para obtener (word, target)
Es una función auxiliar que cambia la estructura del dataset de entrenamiendo de (id, text, target) a (word, target).

In [266]:
# Separo los textos de los tweets en palabras.
def splitTrainIntoWords(dataframe):
    splitted = pd.DataFrame(dataframe['text'].str.split(' ').to_list(), index=dataframe['id']).stack()
    
    # Reestablezco el id.
    splitted = splitted.reset_index([0, 'id'])

    # Le pongo a las segunda columna un nombre descriptivo.
    splitted.columns = ['id','word']

    # Agrego la columna target haciendo un merge con el set original.
    splitted = splitted.merge(dataframe, left_on = 'id', right_on = 'id', how = 'left')

    # Elimino la columna text y id.
    del splitted['text']
    del splitted['id']

    return splitted

## Proceso el test set para obtener (id, word)
Es una función auxiliar que cambia la estructura del dataset de test de (id, text) a (id, word).

In [267]:
# Separo los textos de los tweets en palabras.
def splitTestIntoWords(dataframe):
    # Creo un nuevo dataframe con las palabras separadas.
    splitted = pd.DataFrame(dataframe['text'].str.split(' ').to_list(), index=dataframe['id']).stack()
    
    # Reestablezco el id.
    splitted = splitted.reset_index([0, 'id'])

    # Le pongo a las segunda columna un nombre descriptivo.
    splitted.columns = ['id','word']

    return splitted

## Función de entrenamiento
Esta función toma el set de entrenamiento del formato (word, target) con el cuál se va a realizar el entrenamiento modelo y devuelve un dataframe entrenado del formato (word, target).

In [268]:
def entrenarModelo(train):    
    # Agrupo las palabras y hago un promedio del target.
    trainedDF = train.groupby('word').mean().reset_index()
    
    return trainedDF

## Función de predicción
Esta función realiza las predicciones.

Recibe un set entrenado con el formaro (word, target) y un set de test con el formato (id, word) sobre el cuál realiza las predicciones.

Devuelve un dataframe con las predicciones del tipo (id, predicción).

In [269]:
def predecir(trained, test):    
    # Mergeo el valor del promedio del target de las palabras en este data set.
    testDF = test.merge(trained, left_on = 'word', right_on = 'word', how = 'left')
    
    # Elimino la columna de las palabras.
    del testDF['word']
    
    # Agrupo por id.
    testDF = testDF.groupby('id').mean().reset_index()
    
    # Reestablezco el id.
    testDF = testDF.set_index(['id'])
    
    # Paso los valores a 1 y 0 dependiendo si target > 0.5 o < 0.5.
    testDF['target'] = testDF['target'].apply(lambda x: 1 if x >= 0.5 else 0)
    
    return testDF

## Función para crear submit.
Toma un set de datos de train con formaro (word, target), lo entrena, realiza las predicciones sobre el archivo test con formato (id, word) y guarda un archivo.

In [270]:
def entrenarPredecirYGuardar(train, test, csvFileName):
    trained = entrenarModelo(train)
    predicciones = predecir(trained, test)    
    predicciones.to_csv(csvFileName)

# Creo funciones que van a ser las que se aplican a los datasets para realizar la optimización.

Para mejorar en cada iteración el resultado se crean diferentes funciones que hacen los siguientes cambios:

1 - Se pasan todas las palabras a lowercase.

2 - Se toman los links como si fueran una sola palabra 'http'.

3 - Se quitan los '\n' encontrados en los textos.

4 - Se quitan los stop words.

5 - Se quitan de las palabras todos los caracteres que no sean letras y se eliminan las palabras vacías.

En cada paso se hará un nuevo submit verificando los resultados obtenidos.

## Nomenclatura de los archivos
Se tienen 5 funcionas a aplicar a los datasets, entonces los archivos se nombraran como test-xxxxx.csv.

Si solo se aplica la función 1 entonces el archivo sera test-00001.csv, si no se aplica ninguna función será test-00000.csv y si se aplican la función 1 y la función 3 será test-00101.csv.

## Función para pasar las palabras a minúsculas
Recibe un set de datos con una columna 'word' y devuelve un nuevo set con las palabras en minúscula.

In [271]:
def palabrasALowercase(trained):
    lowercasedDF = trained.copy()
    
    # Paso las palabras a minúsculas.
    lowercasedDF['word'] = lowercasedDF['word'].str.lower()
    
    return lowercasedDF

## Función para tomar los links como una única palabra 'http'
Recibe un set de datos con la columna 'word' y devuelve un nuevo set con los links como si fueran una misma palabra 'http'.

In [272]:
def linksComoHttp(trained):
    httpDF = trained.copy()
    
    # Todos los links pasan a ser la palabra 'http'.
    httpDF.loc[httpDF['word'].str.contains('http', case=True), 'word'] = 'http'
    
    return httpDF

## Función que quita los saltos de línea '\n'
Recibe un set de datos entrenados, un set de test del formato (word, target) o (id, word) y un train set o test set con los la etiquera '\n' eliminada.

In [273]:
def eliminarEtiquetasTrain(train):
    trainDF = train.copy()
    trainDF = pd.DataFrame(trainDF['word'].str.split('\n').to_list(), index=trainDF['target']).stack()
    trainDF = trainDF.reset_index([0, 'target'])
    trainDF.columns = ['target','word']
    trainDF.reset_index()
    
    return trainDF

def eliminarEtiquetasTest(test):
    testDF = test.copy()
    testDF = pd.DataFrame(testDF['word'].str.split('\n').to_list(), index=testDF['id']).stack()
    testDF = testDF.reset_index([0, 'id'])
    testDF.columns = ['id','word']
    testDF.reset_index()
    
    return testDF

## Función que quita todos los caracteres que no sean letras de las palabras.
Dado un dataframe con la columna 'word', elimina caracteres especiales y borra todas las palabras que sean vacías.

In [274]:
def eliminarCharsEspeciales(dataframe): 
    sanDF = dataframe.copy()
    sanDF = sanDF[sanDF['word'].str.isspace() == False]
    sanDF['word'].replace(regex=True, inplace=True, to_replace='[^A-Za-z]', value=r'')
    sanDF = sanDF[sanDF['word'].str.strip().astype(bool)]
    
    return sanDF

## Función para quitar las stop words.
Dado un dataframe con la columna 'word', elimina todos los stop words.

In [275]:
def eliminarStopWords(dataframe):
    stops = stopwords.words('english')
    sinStops = dataframe.copy()
    sinStops = sinStops[~sinStops['word'].isin(stops)]
    return sinStops

# Submits.
A continuación se realizan los submits de prueba.

In [276]:
# 00000.
trainSplitted = splitTrainIntoWords(trainData)
testSplitted = splitTestIntoWords(testData)
entrenarPredecirYGuardar(trainSplitted, testSplitted, 'test-00000.csv')

In [277]:
# 00001.
# Se aplica lowercase a las palabras de los dataframes.
trainLowercased = palabrasALowercase(trainSplitted)
testLowercase = palabrasALowercase(testSplitted)
entrenarPredecirYGuardar(trainLowercased, testLowercase, 'test-00001.csv')

In [278]:
# 00010.
# Se toman los links como la palabra 'http'.
trainHTTP = linksComoHttp(trainSplitted)
testHTTP = linksComoHttp(testSplitted)
entrenarPredecirYGuardar(trainHTTP, testHTTP, 'test-00010.csv')

In [279]:
# 00100.
# Se eliminan las etiquetas como '\n'.
trainSinEtiquetas = eliminarEtiquetasTrain(trainSplitted)
testSinEtiquetas = eliminarEtiquetasTest(testSplitted)
entrenarPredecirYGuardar(trainSinEtiquetas, testSinEtiquetas, 'test-00100.csv')

In [280]:
# 01000.
# Se eliminan las stopwords.
trainStops = eliminarStopWords(trainSplitted)
testStops = eliminarStopWords(testSplitted)

entrenarPredecirYGuardar(trainStops, testStops, 'test-01000.csv')

In [281]:
# 10000.
# Se eliminan los caracteres especiales y las palabras vacías..
trainSan = eliminarCharsEspeciales(trainSplitted)
testSan = eliminarCharsEspeciales(testSplitted)
entrenarPredecirYGuardar(trainSan, testSan, 'test-10000.csv')

In [282]:
# Grid search sobre todas las posibilidades para ver cuál ajusta mejor.
cantidadDeFunciones = 5

for number in range(pow(2, cantidadDeFunciones)):
    bits = [(number >> bit) & 1 for bit in range(cantidadDeFunciones - 1, -1, -1)]
    
    print('----------------')
    print(bits)
    
    if bits[4]:
        print('Aplico función 1')
        
    if bits[3]:
        print('Aplico función 2')
        
    if bits[2]:
        print('Aplico función 3')
        
    if bits[1]:
        print('Aplico función 4')
        
    if bits[0]:
        print('Aplico función 5')
    
    nombreArchivo = "".join([str(a) for a in bits])
    print('nombre del archivo: {}'.format(nombreArchivo))

----------------
[0, 0, 0, 0, 0]
nombre del archivo: 00000
----------------
[0, 0, 0, 0, 1]
Aplico función 1
nombre del archivo: 00001
----------------
[0, 0, 0, 1, 0]
Aplico función 2
nombre del archivo: 00010
----------------
[0, 0, 0, 1, 1]
Aplico función 1
Aplico función 2
nombre del archivo: 00011
----------------
[0, 0, 1, 0, 0]
Aplico función 3
nombre del archivo: 00100
----------------
[0, 0, 1, 0, 1]
Aplico función 1
Aplico función 3
nombre del archivo: 00101
----------------
[0, 0, 1, 1, 0]
Aplico función 2
Aplico función 3
nombre del archivo: 00110
----------------
[0, 0, 1, 1, 1]
Aplico función 1
Aplico función 2
Aplico función 3
nombre del archivo: 00111
----------------
[0, 1, 0, 0, 0]
Aplico función 4
nombre del archivo: 01000
----------------
[0, 1, 0, 0, 1]
Aplico función 1
Aplico función 4
nombre del archivo: 01001
----------------
[0, 1, 0, 1, 0]
Aplico función 2
Aplico función 4
nombre del archivo: 01010
----------------
[0, 1, 0, 1, 1]
Aplico función 1
Aplico funci

In [283]:
trainData.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [284]:
testData.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [285]:
endIndex = round(trainData.shape[0] * 0.1)
#int(round(2.51*100)) 
#trainData[0: end]
endIndex

761

In [286]:
trainData.count()

id        7613
text      7613
target    7613
dtype: int64